In [1]:
import sys
import csv
import time
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn import datasets
import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

from sklearn.metrics import roc_curve, auc

from random import randint
import random
import copy

In [2]:
df=pd.read_csv('/Users/amirasarbaev/Downloads/Internship/data/test/5_time_line.csv',header=0,sep=',')
df['week']=df.loc[:,"week_click":"week_undisp"].min(axis=1)

In [3]:
col=[col for col in df.columns if col not in ["week_click","week_book","week_reply","week_undisp"]]
df=df[col]

# Dataset Split based on displayed offers for target users and timeline

In [4]:
user_list=np.unique(df.user_id)

In [5]:
test=[]
train=[]
for i in tqdm_notebook(range(len(user_list))):
    df_split = np.array_split(df[df.user_id==user_list[i]].sort_values(by='week'), 3)
    train.append(list(df_split[0].index))
    train.append(list(df_split[1].index))
    test.append(list(df_split[2].index))
    
    

A Jupyter Widget

In [6]:
train=list(itertools.chain(*train))
test=list(itertools.chain(*test))

X_train=df[df.index.isin(train)]
X_test=df[df.index.isin(test)]

X_train.set_index(['user_id','item_id'], inplace=True)
X_test.set_index(['user_id','item_id'], inplace=True)

Y_train_click=X_train.click
Y_train_book=X_train.book
Y_train_reply=X_train.reply
Y_test_click=X_test.click
Y_test_book=X_test.book
Y_test_reply=X_test.reply
col=[col for col in df.columns if col not in ['displayed','user_id','item_id',"click","book","reply","week"]]
X_train=X_train[col]
X_test=X_test[col]

# Dataset Normilization

In [7]:
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train.iloc[:,15:])
X_test_minmax = min_max_scaler.transform(X_test.iloc[:,15:])

X_train_minmax = pd.DataFrame(X_train_minmax)
X_test_minmax = pd.DataFrame(X_test_minmax)
X_train_minmax.columns = X_train.iloc[:,15:].columns
X_test_minmax.columns = X_test.iloc[:,15:].columns
X_train_minmax.index = X_train.iloc[:,15:].index
X_test_minmax.index = X_test.iloc[:,15:].index
X_test_minmax

,,career_level_x,discipline_id_x,industry_id_x,country,region_x,experience_n_entries_class,experience_years_experience,experience_years_in_current,edu_degree,career_level_y,discipline_id_y,industry_id_y,country_cat,region_y,employment,active_during_test
user_id,item_id,,,,,,,,,,,,,,,,
16874,16291,0.000000,0.000000,0.000000,0.666667,0.4375,1.000000,0.857143,0.000000,1.000000,0.500000,1.000000,0.652174,0.666667,0.5625,0.2,0.0
24608,16291,0.000000,0.000000,0.347826,0.666667,0.3750,1.000000,1.000000,0.285714,0.000000,0.500000,1.000000,0.652174,0.666667,0.5625,0.2,0.0
27885,16291,0.000000,0.000000,0.652174,0.666667,0.1250,1.000000,1.000000,0.571429,0.000000,0.500000,1.000000,0.652174,0.666667,0.5625,0.2,0.0
33704,16291,0.000000,0.000000,0.913043,0.666667,0.4375,1.000000,1.000000,0.285714,0.000000,0.500000,1.000000,0.652174,0.666667,0.5625,0.2,0.0
41458,16291,0.500000,0.000000,0.000000,0.666667,0.0000,1.000000,0.857143,0.142857,0.666667,0.500000,1.000000,0.652174,0.666667,0.5625,0.2,0.0
57649,16291,1.000000,0.826087,0.608696,0.000000,0.0000,1.000000,1.000000,0.285714,0.000000,0.500000,1.000000,0.652174,0.666667,0.5625,0.2,0.0
65409,16291,0.000000,0.000000,0.000000,0.666667,0.0000,1.000000,1.000000,0.428571,0.000000,0.500000,1.000000,0.652174,0.666667,0.5625,0.2,0.0
93937,16291,0.666667,0.652174,0.043478,0.666667,0.0000,1.000000,1.000000,0.428571,0.000000,0.500000,1.000000,0.652174,0.666667,0.5625,0.2,0.0
98519,16291,0.000000,0.000000,0.000000,0.666667,0.3750,1.000000,0.000000,0.000000,0.000000,0.500000,1.000000,0.652174,0.666667,0.5625,0.2,0.0


In [8]:
sc=StandardScaler()
sc.fit(X_train_minmax)
X_train_std=sc.transform(X_train_minmax)
X_test_std=sc.transform(X_test_minmax)

X_train_std = pd.DataFrame(X_train_std)
X_test_std = pd.DataFrame(X_test_std)
X_train_std.columns = X_train.iloc[:,15:].columns
X_test_std.columns = X_test.iloc[:,15:].columns
X_train_std.index = X_train.iloc[:,15:].index
X_test_std.index = X_test.iloc[:,15:].index
X_test_std

,,career_level_x,discipline_id_x,industry_id_x,country,region_x,experience_n_entries_class,experience_years_experience,experience_years_in_current,edu_degree,career_level_y,discipline_id_y,industry_id_y,country_cat,region_y,employment,active_during_test
user_id,item_id,,,,,,,,,,,,,,,,
16874,16291,-1.166280,-0.491644,-0.784702,0.064055,0.980465,0.629910,0.594784,-1.310319,2.208837,0.389085,1.735483,0.421063,0.298260,1.239454,-0.118256,-0.528653
24608,16291,-1.166280,-0.491644,0.235617,0.064055,0.717546,0.629910,1.132086,-0.002516,-0.879458,0.389085,1.735483,0.421063,0.298260,1.239454,-0.118256,-0.528653
27885,16291,-1.166280,-0.491644,1.128396,0.064055,-0.334130,0.629910,1.132086,1.305287,-0.879458,0.389085,1.735483,0.421063,0.298260,1.239454,-0.118256,-0.528653
33704,16291,-1.166280,-0.491644,1.893636,0.064055,0.980465,0.629910,1.132086,-0.002516,-0.879458,0.389085,1.735483,0.421063,0.298260,1.239454,-0.118256,-0.528653
41458,16291,0.323988,-0.491644,-0.784702,0.064055,-0.859968,0.629910,0.594784,-0.656417,1.179405,0.389085,1.735483,0.421063,0.298260,1.239454,-0.118256,-0.528653
57649,16291,1.814255,3.234819,1.000856,-6.304858,-0.859968,0.629910,1.132086,-0.002516,-0.879458,0.389085,1.735483,0.421063,0.298260,1.239454,-0.118256,-0.528653
65409,16291,-1.166280,-0.491644,-0.784702,0.064055,-0.859968,0.629910,1.132086,0.651386,-0.879458,0.389085,1.735483,0.421063,0.298260,1.239454,-0.118256,-0.528653
93937,16291,0.820743,2.450300,-0.657162,0.064055,-0.859968,0.629910,1.132086,0.651386,-0.879458,0.389085,1.735483,0.421063,0.298260,1.239454,-0.118256,-0.528653
98519,16291,-1.166280,-0.491644,-0.784702,0.064055,0.717546,0.629910,-2.629023,-1.310319,-0.879458,0.389085,1.735483,0.421063,0.298260,1.239454,-0.118256,-0.528653


# Union of extracted features + normolized contextual

In [ ]:
U=np.unique(X_train_std.reset_index().user_id)
I=np.unique(X_train_std.reset_index().item_id)

In [ ]:
from random import randint
import random
import copy
def sample_floats(low, high, k=1):
    """ Return a k-length list of unique random floats
        in the range of low <= x <= high
    """
    result = []
    seen = set()
    for i in range(k):
        x = random.uniform(low, high)
        while x in seen:
            x = random.uniform(low, high)
        seen.add(x)
        result.append(x)
    return result

In [ ]:
U_repr = {U[i]:[v for v in sample_floats(0,1,k=X_train.shape[1])] for i in range (len(U))}
I_repr= {I[j]:[w for w in sample_floats(0,1,k=X_train.shape[1])] for j in range (len(I))}


In [ ]:
Y_train_click=Y_train_click.reset_index()
Y_train_book=Y_train_book.reset_index()
Y_train_reply=Y_train_reply.reset_index()
X_train_std=X_train_std.reset_index()
X_test_std=X_test_std.reset_index()
Y_test_click=Y_test_click.reset_index()
Y_test_book=Y_test_book.reset_index()
Y_test_reply=Y_test_reply.reset_index()

In [ ]:
X_train_dict={}
for i in tqdm_notebook(xrange(len(X_train_std))):
    X_train_dict[X_train_std.user_id[i],X_train_std.item_id[i]]=X_train.iloc[i,:15].values.tolist()+\
    X_train_std.iloc[i,2:].values.tolist()
#     +U_repr[X_train_std.user_id[i]]+I_repr[X_train_std.item_id[i]]

In [ ]:
Y_train_dict_click={}
Y_train_dict_book={}
Y_train_dict_reply={}
for i in tqdm_notebook(xrange(len(Y_train_click))):
    Y_train_dict_click[Y_train_click.user_id[i],Y_train_click.item_id[i]]=Y_train_click.iloc[i,2]
    Y_train_dict_book[Y_train_book.user_id[i],Y_train_book.item_id[i]]=Y_train_book.iloc[i,2]
    Y_train_dict_reply[Y_train_reply.user_id[i],Y_train_reply.item_id[i]]=Y_train_reply.iloc[i,2]
    

# Stochastic gradient descent

In [ ]:
indx_dict={}
for i in range(len(U)):
    indx_dict[U[i]]=X_train_std[X_train_std.user_id==U[i]].iloc[:,:2].values.tolist()

In [ ]:
#Inputs:
eta=0.0001
eps=1e-3
lamb=1e-3
nb_runs = 600000
#Initialization:

global_loss = [1,0]
nb=1
while abs(global_loss[nb-1]-global_loss[nb])>eps:
    loss_local=0
    for j in tqdm_notebook(range(nb_runs)):
        u = randint(0,len(U)-1)
        
        target_samples=indx_dict[U[u]]
        target_lables=[Y_train_dict_click[tuple(elem)] for elem in target_samples]
        positive=[target_samples[m] for m in [l for l, e in enumerate(target_lables) if e != 0]]
        negative=[target_samples[m] for m in [l for l, e in enumerate(target_lables) if e == 0]]

        if (len(positive)!=0 and len(negative)!=0): 

            p=randint(0, len(positive)-1)
            n=randint(0, len(negative)-1)

            user=U_repr[U[u]]
            item_plus = I_repr[positive[p][1]]
            item_minus = I_repr[negative[n][1]]
            
            diff=(np.array(item_minus) - np.array(item_plus)).tolist()
            ex=np.exp(np.dot(user,diff))
            dl_4_user =  diff / ( 1 + (1. / ex ) )
            dl_4_item_plus = ( (-1) * np.array( user ) ) / ( 1 + (1. / ex ) )
            dl_4_item_minus = ( user ) / ( 1 + (1. / ex ) )

                
            user = user - eta * ( ( dl_4_user ) + ( 2 * lamb * np.sum( user ) ) ) 

            item_plus = item_plus - eta * ( ( dl_4_item_plus ) + ( 2 * lamb * np.sum( item_plus ) ) )

            item_minus = item_minus - eta * ( ( dl_4_item_minus ) + ( 2 * lamb *np.sum( item_minus ) ) )

            loss_local=loss_local+(np.log( 1. + ex))+\
            lamb*(np.linalg.norm(user,2)**2+np.linalg.norm(item_plus,2)**2+np.linalg.norm(item_minus,2)**2)

            U_repr[U[u]]=user.tolist()
            I_repr[positive[p][1]]=item_plus.tolist()
            I_repr[negative[n][1]]=item_minus.tolist()

    global_loss.append(loss_local/nb_runs)
    nb+=1
    print(global_loss)

        
        


# Ranking

In [ ]:
U_pandas=pd.DataFrame(U_repr.items())
I_pandas=pd.DataFrame(I_repr.items())

In [ ]:
U_pandas=U_pandas.set_index(0)
U_pandas.index.name="user_id"
I_pandas=I_pandas.set_index(0)
I_pandas.index.name="item_id"

In [ ]:
U_pandas=U_pandas[1].apply(pd.Series)
cols=['U_'+str(i) for i in range (31)]
U_pandas.columns=cols

In [ ]:
I_pandas=I_pandas[1].apply(pd.Series)
cols=['I_'+str(i) for i in range (31)]
I_pandas.columns=cols

In [ ]:
X_final=X_test_std

In [ ]:
I_test=np.unique(X_test_std.item_id)
main_list = np.setdiff1d(I_test,I)
random_items={}
for i in range(len (main_list)):
    random_items[main_list[i]]=[v for v in sample_floats(0,1,k=31)]
    
    
    

In [ ]:
pd_scores=[]
for i in tqdm_notebook(range(len(X_final))):
    try:
        pd_scores.append(np.dot(U_pandas[U_pandas.index==X_final.iloc[i,0]].values.tolist()[0],\
                    I_pandas[I_pandas.index==X_final.iloc[i,1]].values.tolist()[0]))
    except:
        pd_scores.append(np.dot(U_pandas[U_pandas.index==X_final.iloc[i,0]].values.tolist()[0],\
                    random_items[X_final.iloc[i,1]]))

In [ ]:
X_final['scores']=pd_scores
X_final=pd.merge(X_final,Y_test_click,how='left',on=['user_id','item_id'])

In [ ]:
k=5
total_map=[]
for i in tqdm_notebook(range (len(U))):
    target_list=X_final[X_final.user_id==U[i]].sort_values(by='scores',ascending=False)
    #     target_list=target_list[target_list.Y_test==1]
    up=1
    down=1
    precision=[]
    for j,score in enumerate(target_list.click[:k].values.tolist()):
        if score!=0:
            precision.append(float(up)/float(down+j))
            up+=1
        else:
            precision.append(0)
    total_map.append(np.sum(precision[0:k])/k)
np.mean(total_map)

# Classification

In [ ]:
Class_train={}
for i in tqdm_notebook(range(len(X_train_std))):
        Class_train[X_train_std.user_id[i],X_train_std.item_id[i]]=U_repr[X_train_std.user_id[i]]+\
        I_repr[X_train_std.item_id[i]]
#         +list(Y_train_click[(Y_train_click.user_id==X_train_std.user_id[i])&(Y_train_click.item_id==X_train_std.item_id[i])].click)


In [ ]:
Class_train=pd.DataFrame(Class_train.items())
col=["U_repr_"+str(i) for i in range(31)]+["I_repr_"+str(i) for i in range(31)]

In [ ]:
Class_train[['user_id','item_id']] = pd.DataFrame(Class_train[0].values.tolist(), index= Class_train.index)

In [ ]:
Class_train[['user_id','item_id']] = pd.DataFrame(Class_train[0].values.tolist(), index= Class_train.index)
Class_train[col] = pd.DataFrame(Class_train[1].values.tolist(), index= Class_train.index)
Class_train.iloc[:,4:]

In [ ]:
Class_train=pd.merge(Class_train,Y_train_click,how="left",on=["user_id","item_id"])

In [ ]:
lr = LogisticRegressionCV(penalty='l2',class_weight='balanced')
lr.fit(Class_train.iloc[:,4:-1], Class_train.iloc[:,-1]) 

In [ ]:
Y_test_click_dic={}
for i in tqdm_notebook(xrange(len(Y_test_click))):
    Y_test_click_dic[Y_test_click.user_id[i],Y_test_click.item_id[i]]=Y_test_click.iloc[i,2]
    

In [ ]:
X_final=X_test_std

In [ ]:
Class={}
for i in tqdm_notebook(range(len(X_final))):
    try:
        Class[X_final.user_id[i],X_final.item_id[i]]=U_repr[X_final.user_id[i]]+\
        I_repr[X_final.item_id[i]]+\
        [Y_test_click_dic[X_final.user_id[i],X_final.item_id[i]]]
    except:
        Class[X_final.user_id[i],X_final.item_id[i]]=U_repr[X_final.user_id[i]]+\
        random_items[X_final.item_id.values.tolist()[i]]+\
        [Y_test_click_dic[X_final.user_id[i],X_final.item_id[i]]]
       

In [ ]:
col=col+['Y_test_click']

In [ ]:
Class_final=pd.DataFrame(Class.items())
Class_final[['user_id','item_id']] = pd.DataFrame(Class_final[0].values.tolist(), index= Class_final.index)
Class_final[col] = pd.DataFrame(Class_final[1].values.tolist(), index= Class_final.index)
Class_final

In [ ]:
Y_pred=lr.predict(Class_final.iloc[:,4:-1])
print(classification_report(Class_final.iloc[:,-1], Y_pred))
confusion_matrix(Class_final.iloc[:,-1], Y_pred)

In [ ]:
false_positive_rate, recall, thresholds = roc_curve(Y_pred,Class_final.iloc[:,-1])
roc_auc = auc(false_positive_rate, recall)
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, recall, 'b', label='AUC = %0.2f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.ylabel('Recall')
plt.xlabel('Fall-out')
plt.show()